In [1]:
import tensorflow as tf
import random
import sys
# ----------- change the paths according to your environment -----------
sys.path.append('.')
import mypreprocessing
from generator import DataGenerator
from models import getNetwork
from custom_layers import MeanOverTime
import sys, os
import json
import scipy
import keras
import tensorflow as tf
from keras import optimizers, initializers, regularizers, constraints
from utils import *
from sklearn import metrics
import matplotlib as plt
from keras.layers import *
from tcn import TCN
from keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyp
from keras.layers import Layer
from sklearn.metrics import classification_report

class ChannelAttention(Layer):
    def __init__(self, ratio = 8):
        super(ChannelAttention, self).__init__()
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]

        self.dense = tf.keras.layers.Dense(units=self.channels // self.ratio,
                                           activation='relu',
                                           kernel_initializer='he_normal',
                                           use_bias=True)
        self.dense_output = tf.keras.layers.Dense(units=self.channels,
                                                  kernel_initializer='he_normal',
                                                  use_bias=True)

    def call(self, inputs):
        # inputs = tf.expand_dims(inputs, axis=2)
        gap = tf.reduce_mean(inputs, axis=[0, 1], keepdims=True)

        attention = self.dense(gap)
        attention = self.dense_output(attention)
        attention = tf.keras.activations.sigmoid(attention)

        output = tf.multiply(inputs, attention)

        return output

tf.random.set_seed(42)
print('OK')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

os.getcwd()

folder_process_data = './DB2'

os.environ['PYTHONHASHSEED'] = '0'

print('Keras:', keras.__version__)
print('Tensorflow:', tf.__version__)

# 1. Logging
'''
if len(sys.argv) == 4:
    CONFIG_FILE = str(sys.argv[1])
    SUBJECT = int(sys.argv[2])
    TIMESTAMP = int(sys.argv[3])
else:
    print('Expected different number of arguments. {} were given'.format(len(sys.argv) - 1))
    sys.exit()
    '''
# ----------- change the paths according to your environment -----------
CONFIG_FILE='./config_DB2/TCCNet_aot_2500.json'
import datetime
from datetime import date
SUBJECT=1
GESTURES=50
currentDateAndTime = datetime.datetime.now()

current_date = datetime.datetime.now()
TIMESTAMP = current_date .strftime("%Y%m%d%H%M%S")

# ----------- change the paths according to your environment -----------
with open(CONFIG_FILE) as json_file:
    config_params = json.load(json_file)
outdir='./Result'
LOGGING_FILE_PREFIX = config_params['logging']['log_file'] + '_' + str(TIMESTAMP)
if config_params['logging']['enable']:
    LOGGING_FILE = './Result/L_' + LOGGING_FILE_PREFIX + '.log'# 'Musa_NinaPro_Project/ProcessedData/DB1/Result/L_' + LOGGING_FILE_PREFIX + '.log'
    LOGGING_TENSORBOARD_FILE = './Result/tblogs/L_' + LOGGING_FILE_PREFIX #'Musa_NinaPro_Project/ProcessedData/DB1/Result/tblogs/L_' + LOGGING_FILE_PREFIX

if config_params['model']['save']:
    MODEL_SAVE_FILE = './Result/models/O1_' + LOGGING_FILE_PREFIX + '_{}.json'
    MODEL_WEIGHTS_SAVE_FILE = './Result/models/O2_' + LOGGING_FILE_PREFIX + '_{}.h5'

METRICS_SAVE_FILE = './Result/metrics/O3_' + LOGGING_FILE_PREFIX + '_{}.mat'

if not os.path.exists(os.path.dirname(METRICS_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(METRICS_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(MODEL_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(MODEL_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(LOGGING_TENSORBOARD_FILE)):
    try:
        os.makedirs(os.path.dirname(LOGGING_TENSORBOARD_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise


print('Logging file: {}'.format(LOGGING_FILE))
print('Tensorboard file: {}'.format(LOGGING_TENSORBOARD_FILE))
print('Model JSON file: {}'.format(MODEL_SAVE_FILE))
print('Model H5 file: {}'.format(MODEL_WEIGHTS_SAVE_FILE))
print('Metrics file: {}'.format(METRICS_SAVE_FILE))

# 2. Config params generator
PARAMS_TRAIN_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('train_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_TRAIN_GENERATOR[key] = params_gen[key]

PARAMS_VALID_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('valid_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_VALID_GENERATOR[key] = params_gen[key]

2025-04-15 08:03:05.103318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 08:03:05.818461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OK
1 Physical GPUs, 1 Logical GPUs
Keras: 2.12.0
Tensorflow: 2.12.0
Logging file: ./Result/L_TCCNet_att_20250415080309.log
Tensorboard file: ./Result/tblogs/L_TCCNet_att_20250415080309
Model JSON file: ./Result/models/O1_TCCNet_att_20250415080309_{}.json
Model H5 file: ./Result/models/O2_TCCNet_att_20250415080309_{}.h5
Metrics file: ./Result/metrics/O3_TCCNet_att_20250415080309_{}.mat


2025-04-15 08:03:09.087236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 08:03:09.100674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 08:03:09.100740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 08:03:09.104840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 08:03:09.104898: I tensorflow/compile

In [2]:
target_subject = 3

In [3]:
import tensorflow as tf
import random
import mypreprocessing
from DB2_Mydataset import *
from models import getNetwork
import keras
from keras import optimizers, initializers, regularizers, constraints
from utils import *
from sklearn import metrics
import os
import numpy as np

# Environment settings
folder_process_data = './DB2'
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1234)
random.seed(12345)

# GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Set random seed
tf.random.set_seed(1234)

# Logging settings
SUBJECT = 1
GESTURES = 50

# Create save directories if they don't exist
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

create_directory_if_not_exists(os.path.dirname(METRICS_SAVE_FILE))
create_directory_if_not_exists(os.path.dirname(MODEL_SAVE_FILE))
create_directory_if_not_exists(os.path.dirname(LOGGING_TENSORBOARD_FILE))

# Initialize parameters
PARAMS_TRAIN_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('train_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_TRAIN_GENERATOR[key] = params_gen[key]

PARAMS_VALID_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('valid_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_VALID_GENERATOR[key] = params_gen[key]

# Prepare data
INPUT_DIRECTORY = folder_process_data
PARAMS_TRAIN_GENERATOR.update({
    'preprocess_function_1': [mypreprocessing.lpf],
    'preprocess_function_1_extra': [{'fs': 100}],
    'data_type': 'rms',
    'classes': [i for i in range(50)],
    'dim': [1, 12],
    'repetitions': [1, 3, 4, 6],
    'min_max_norm': True,
    'batch_size': 64
})

PARAMS_VALID_GENERATOR.update({
    'preprocess_function_1': [mypreprocessing.lpf],
    'preprocess_function_1_extra': [{'fs': 100}],
    'data_type': 'rms',
    'classes': [i for i in range(50)],
    'dim': [1, 12],
    'repetitions': [2, 5],
    'min_max_norm': True,
    'batch_size': 64
})

# Remove 'input_directory' key
PARAMS_TRAIN_GENERATOR.pop('input_directory', None)
PARAMS_VALID_GENERATOR.pop('input_directory', None)

# Load data
input_dir = f'{INPUT_DIRECTORY}/subject-{SUBJECT:02d}'
train_generator = DataGenerator(input_directory=input_dir, **PARAMS_TRAIN_GENERATOR)
valid_generator = DataGenerator(input_directory=input_dir, **PARAMS_VALID_GENERATOR)

X_train, Y_train, train_reps, L_ = train_generator.get_data()
y_train = np.where(Y_train == 1)[1]

X_test, Y_test, test_reps = valid_generator.get_data()
y_test = np.where(Y_test == 1)[1]

# Reshape data
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
train_dataset = Hand_Dataset(X_train, y_train)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
test_dataset = Hand_Dataset(X_test, y_test)

class_num = train_generator.n_classes
timelen = X_train.shape[1]

print(f'Now the data from subject {format(SUBJECT)} have loaded.')

Now the data from subject 1 have loaded.


In [4]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import optimizers
from keras.layers import *
from keras.models import Model
from keras.callbacks import Callback
from tcn import TCN
from sklearn.metrics import classification_report, accuracy_score
import scipy.io

# Channel Attention Layer
class ChannelAttention(Layer):
    def __init__(self, ratio=8, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]
        self.dense = Dense(units=self.channels // self.ratio,
                           activation='relu',
                           kernel_initializer='he_normal',
                           use_bias=True)
        self.dense_output = Dense(units=self.channels,
                                  kernel_initializer='he_normal',
                                  use_bias=True)

    def call(self, inputs):
        gap = tf.reduce_mean(inputs, axis=2, keepdims=True)
        attention = self.dense(gap)
        attention = self.dense_output(attention)
        attention = tf.keras.activations.sigmoid(attention)
        return inputs * attention, attention  # Two outputs!

# SE Block
def se_block(in_block, ch, ratio=16):
    z = GlobalAveragePooling1D()(in_block)
    x = Dense(ch // ratio, activation='relu')(z)
    x = Dense(ch, activation='sigmoid')(x)
    return multiply([in_block, x])

# Build the model
columns = 12
gestures = 50
input = Input(shape=(None, columns))  # (time, 12)

# Build each branch
branch1 = TCN(nb_filters=12, nb_stacks=1, kernel_size=3, padding='same', return_sequences=True)(input)
branch1 = Bidirectional(LSTM(64, return_sequences=True))(branch1)

branch2 = Conv1D(16, 3, padding='same')(input)
branch2 = SeparableConv1D(filters=16, kernel_size=3, padding='same', activation='relu')(branch2)
branch2 = se_block(branch2, ch=16)

branch3 = Bidirectional(TCN(nb_filters=32, nb_stacks=1, kernel_size=3, padding='same', return_sequences=True))(input)

# Merge all branches
x = concatenate([branch1, branch2, branch3])  # shape: (batch, time, channels)
x = Dropout(0.2)(x)
x = Conv1D(12, kernel_size=1, padding='same')(x)  # shape: (batch, time, 12)
x, attention_scores = ChannelAttention()(x)  # Apply ChannelAttention

# Dimensionality reduction with GlobalAveragePooling
x = GlobalAveragePooling1D()(x)
output = Dense(gestures, activation='softmax')(x)

model = Model(inputs=input, outputs=[output, attention_scores])  # Note: two outputs

# Save predictions and attention scores after each epoch
class SavePredictionsCallback(Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        self.predictions_per_epoch = []
        self.true_labels_per_epoch = []
        self.attention_scores_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred, attention_scores = self.model.predict(self.X_test)
        self.predictions_per_epoch.append(y_pred)
        self.true_labels_per_epoch.append(self.y_test)
        self.attention_scores_per_epoch.append(attention_scores)
        print(f"Epoch {epoch+1}: saved predictions and attention scores")

train_callbacks = []

# Add to callbacks
save_predictions_callback = SavePredictionsCallback(X_test, Y_test)
train_callbacks.append(save_predictions_callback)

# Optimizer setup
learning_rate = 0.001
config_params = {'training': {'optimizer': 'adam'}}  # Example config
if config_params['training']['optimizer'] == 'adam':
    optimizer = optimizers.Adam(learning_rate=learning_rate, epsilon=0.001)
elif config_params['training']['optimizer'] == 'sgd':
    optimizer = optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Define metrics (example: top-3 and top-5 accuracy)
def top_3_accuracy(y_true, y_pred):
    return tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_5_accuracy(y_true, y_pred):
    return tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=['categorical_crossentropy', None],  # Use categorical crossentropy for classification
    metrics=[['accuracy', top_3_accuracy, top_5_accuracy], []]  # Define the metrics
)

# Train the model
history = model.fit(train_generator, epochs=2,
                    validation_data=(X_test, Y_test),
                    callbacks=train_callbacks, verbose=2)

# Save the results from the final epoch
final_attention = save_predictions_callback.attention_scores_per_epoch[-1]
final_labels = save_predictions_callback.true_labels_per_epoch[-1]

# attention_scores shape: (samples, time_steps, channels)
# Save with time_steps as a dimension
scipy.io.savemat('final_attention_scores.mat', {
    'attention_scores': final_attention,  # (samples, time_steps, channels)
    'labels': final_labels                # (samples, num_classes)
})
print("Saved attention scores and labels to 'final_attention_scores.mat'")

2025-04-15 08:03:50.186012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-04-15 08:03:50.187588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-04-15 08:03:50.188604: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/2


2025-04-15 08:03:51.625238: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-04-15 08:03:52.059573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-04-15 08:03:52.061069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

2/2 [==============================] - 2s 427ms/step
Epoch 1: saved predictions and attention scores
40/40 - 127s - loss: 6.0693 - dense_2_loss: 6.0693 - dense_2_accuracy: 0.0230 - dense_2_top_3_accuracy: 0.0785 - dense_2_top_5_accuracy: 0.1285 - val_loss: 3.8811 - val_dense_2_loss: 3.8811 - val_dense_2_accuracy: 0.0000e+00 - val_dense_2_top_3_accuracy: 0.1406 - val_dense_2_top_5_accuracy: 0.2500 - 127s/epoch - 3s/step
Epoch 2/2
2/2 [==============================] - 1s 463ms/step
Epoch 2: saved predictions and attention scores
40/40 - 69s - loss: 3.8971 - dense_2_loss: 3.8971 - dense_2_accuracy: 0.0473 - dense_2_top_3_accuracy: 0.1117 - dense_2_top_5_accuracy: 0.1594 - val_loss: 3.8333 - val_dense_2_loss: 3.8333 - val_dense_2_accuracy: 0.0781 - val_dense_2_top_3_accuracy: 0.1562 - val_dense_2_top_5_accuracy: 0.2500 - 69s/epoch - 2s/step
Saved attention scores and labels to 'final_attention_scores.mat'


In [5]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
import os

# Load .mat file
data = scipy.io.loadmat('final_attention_scores.mat')
attention_scores = data['attention_scores']  # shape: (samples, time_steps, channels)
labels_onehot = data['labels']               # shape: (samples, num_classes)
labels = np.argmax(labels_onehot, axis=1)    # Convert to class indices

num_classes = np.max(labels) + 1
time_steps = attention_scores.shape[1]
num_channels = attention_scores.shape[2]

summary_table = []

# Create output folder
os.makedirs("class_csv", exist_ok=True)

for cls in range(num_classes):
    idx = np.where(labels == cls)[0]
    if len(idx) == 0:
        print(f"Class {cls+1} has no samples.")
        continue

    class_attention = attention_scores[idx]  # shape: (samples_in_class, time_steps, channels)
    mean_attention = np.mean(class_attention, axis=0)  # shape: (time_steps, channels)

    # ---- Plot ----
    plt.figure(figsize=(12, 7))
    for ch in range(min(num_channels, 12)):
        plt.plot(mean_attention[:, ch], label=f'Ch{ch+1}')
    plt.title(f'Average Attention Scores (Class {cls+1})')
    plt.xlabel('Time step')
    plt.ylabel('Attention Score')
    plt.legend(loc='upper right', ncol=2)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'class_{cls+1:02d}_attention_12ch.png')
    plt.close()

    # ---- Save CSV (horizontal format) ----
    df_class = pd.DataFrame(mean_attention[:, :12], columns=[f'Ch{i+1}' for i in range(12)])
    df_class.insert(0, 'TimeStep', np.arange(time_steps))  # Insert TimeStep column at the beginning
    df_class.to_csv(f'class_csv/class{cls+1:02d}_12ch.csv', index=False)

    # ---- Compute overall mean score ----
    overall_mean = np.mean(mean_attention[:, :12])
    summary_table.append([cls+1, overall_mean])

# ---- Save summary ----
df_summary = pd.DataFrame(summary_table, columns=['Class', 'Mean_Attention'])
df_summary.to_csv('attention_summary_12ch.csv', index=False)

print("✅ Done: Saved 1 file per class in horizontal format (TimeStep + 12 channels) under 'class_csv' folder.")

Class 1 has no samples.
✅ Done: Saved 1 file per class in horizontal format (TimeStep + 12 channels) under 'class_csv' folder.


In [6]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd

# Load .mat file
data = scipy.io.loadmat('final_attention_scores.mat')
attention_scores = data['attention_scores']  # shape: (samples, time_steps, channels)
labels_onehot = data['labels']               # shape: (samples, num_classes)
labels = np.argmax(labels_onehot, axis=1)    # Convert one-hot to class indices

# ---- ✅ Average across all channels to get a single-channel attention score ----
attention_scores = np.mean(attention_scores, axis=2, keepdims=True)  # shape: (samples, time_steps, 1)

# ---- ✅ Cut off the first 400 time steps ----
attention_scores = attention_scores[:, 400:, :]  # shape: (samples, time_steps - 400, 1)

num_classes = np.max(labels) + 1
time_steps = attention_scores.shape[1]

summary_table = []

for cls in range(num_classes):
    idx = np.where(labels == cls)[0]
    if len(idx) == 0:
        print(f"Class {cls+1} has no samples.")
        continue

    class_attention = attention_scores[idx]  # shape: (samples_in_class, time_steps, 1)
    mean_attention = np.mean(class_attention, axis=0)  # shape: (time_steps, 1)

    # ---- Plot ----
    plt.figure(figsize=(10, 6))
    plt.plot(mean_attention[:, 0], label='AvgCh')  # Only 1 channel
    plt.title(f'Average Attention Scores (Class {cls+1})')
    plt.xlabel('Time step (after cutting first 400)')
    plt.ylabel('Attention Score')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'class_{cls+1:02d}_ave_attention.png')
    plt.close()

    # ---- Save overall mean attention score (mean across time steps) ----
    overall_mean = np.mean(mean_attention)
    summary_table.append([cls+1, overall_mean])

# ---- Save summary as CSV ----
df = pd.DataFrame(summary_table, columns=['Class', 'Mean_Attention'])
df.to_csv('attention_summary.csv', index=False)

print("✅ Done: Averaged 12 channels → cut first 400 time steps → plotted & saved to CSV.")

Class 1 has no samples.
✅ Done: Averaged 12 channels → cut first 400 time steps → plotted & saved to CSV.
